In [1]:
import dash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash


# LOAD SPACEX DATA 
spacex_df = pd.read_csv("spacex_launch_geo.csv")
max_payload = spacex_df['PAYLOAD_MASS_KG_'].max()
min_payload = spacex_df['PAYLOAD_MASS_KG_'].min()


# CREATE DASH APPLICATION
app = dash.Dash(__name__)

# CREATE DASH LAYOUT
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                             value='ALL', 
                                             placeholder="State",
                                             searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                
                                # TASK 3: Add a Range Slider to Select Payload
                                html.Div(id="sentence-output", children=["Paylaod range(kg):"], style={}),
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 100: '100'},
                                                value=['min_payload', 'max_payload']),

                                # TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):    
    if entered_site == 'ALL':
        fig_pie = px.pie(spacex_df, 
                     values='class', 
                     names='Launch_Site', 
                     title='Total Success Launches By Site')        
    else:
        filtered_df = spacex_df[spacex_df['Launch_Site'] == entered_site]
        filtered_df = filtered_df.groupby('class').count().reset_index()        
        fig_pie = px.pie(filtered_df, 
                     values='Unnamed: 0', 
                     names='class', 
                     title='Total Launches for site {}'.format(entered_site))        
        # return the outcomes piechart for a selected site
    return fig_pie
        
# TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
            [Output(component_id='success-payload-scatter-chart', component_property='figure'), Output(component_id='sentence-output', component_property='style')],
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value")],
              prevent_initial_call=False)

def get_scatter_chart(entered_site, payload_range):
    print('Params: {} {}'.format(entered_site, payload_range))
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['PAYLOAD_MASS_KG_'] >= int(payload_range[0])) &
                                (spacex_df['PAYLOAD_MASS_KG_'] <= int(payload_range[1]))
                               ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='All sites - payload mass between {:8,d}kg and {:8,d}kg'.format(int(payload_range[0]),int(payload_range[1])))
    else:
        filtered_df = spacex_df[(spacex_df['Launch_Site'] == entered_site) & 
                                (spacex_df['PAYLOAD_MASS_KG_'] >= int(payload_range[0])) &
                                (spacex_df['PAYLOAD_MASS_KG_'] <= int(payload_range[1]))
                               ]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Site {} - payload mass between {:8,d}kg and {:8,d}kg'.format(entered_site,int(payload_range[0]),int(payload_range[1])))
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=9093)

ImportError: cannot import name 'escape' from 'jinja2' (/Users/furnivk/miniconda3/lib/python3.11/site-packages/jinja2/__init__.py)

In [7]:
import plotly.express as px
import pandas as pd

In [4]:
spacex_df = pd.read_csv("data/spacex.csv")
spacex_df.head(5)

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,4/6/10,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,8/12/10,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,8/10/12,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,1/3/13,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [ ]:
# fig = px.pie(spacex_df, values='Launch_Site', names='Orbit')
# fig.show()